## Sentiment analyse twitter

In [42]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import re

### We load an explore our labeled dataset 

labeld training set can be download from:http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

In [43]:
##This chunck is for reading in the train and test data, including a little bit of exploratory.

cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv("Datafiles\\training.1600000.processed.noemoticon.csv",header=None, names=cols,encoding="ISO-8859–1")
# above line will be different depending on where you saved your data, and your file name
df.head()
df.drop(['id','date','query_string','user'],axis=1,inplace=True) ## Drop the rows that we dont need
df[df.sentiment == 4].head(10) ## Show some rows of sentiment 4 , looks like 4 is positive


,sentiment,text
800000,4,I LOVE @Health4UandPets u guys r the best!!
800001,4,im meeting up with one of my besties tonight! Cant wait!! - GIRL TALK!!
800002,4,"@DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart."
800003,4,"Being sick can be really cheap when it hurts too much to eat real food Plus, your friends make you soup"
800004,4,@LovesBrooklyn2 he has that effect on everyone
800005,4,@ProductOfFear You can tell him that I just burst out laughing really loud because of that Thanks for making me come out of my sulk!
800006,4,@r_keith_hill Thans for your response. Ihad already find this answer
800007,4,"@KeepinUpWKris I am so jealous, hope you had a great time in vegas! how did you like the ACM's?! LOVE YOUR SHOW!!"
800008,4,"@tommcfly ah, congrats mr fletcher for finally joining twitter"
800009,4,@e4VoIP I RESPONDED Stupid cat is helping me type. Forgive errors


In [44]:
df[df.sentiment == 0].head(10) ## Show some rows of sentiment 0, looks like 0 is negative

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
5,0,@Kwesidei not the whole crew
6,0,Need a hug
7,0,"@LOLTrish hey long time no see! Yes.. Rains a bit ,only a bit LOL , I'm fine thanks , how's you ?"
8,0,@Tatiana_K nope they didn't have it
9,0,@twittera que me muera ?


In [45]:
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1}) ## Map indicater 4 to 1
df.sentiment.value_counts() ## Check how much labeled is 800k sentiment 4 and 800k sentiment 0

1    800000
0    800000
Name: sentiment, dtype: int64

## We load our untagged dataset

In [46]:
%%time
# Verander in de regel hieronder de locatie van de data
tweets = pd.read_json("Datafiles\\mini_twitter_geotagged_clean.json", orient='records', lines=True)


Wall time: 9.96 s


In [47]:
tweets['ID'] = tweets.index ## Performance reasons and slicing for myself
tweets = tweets[['ID','id','CleanText','text','entities','in_reply_to_status_id', 'quoted_status_id']]
tweets.head()

,ID,id,CleanText,text,entities,in_reply_to_status_id,quoted_status_id
0,0,764039733076897792,\n\n\nALL IN COLLUSION TOGETHER \n\n\n\n\n\n\nhttpstco5GMNZq40V3,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3,"{'hashtags': [{'text': 'NOJUSTICE', 'indices': [62, 72]}, {'text': 'TrumpPence', 'indices': [93, 104]}], 'urls': [{'url': 'https://t.co/5GMNZq40V3', 'expanded_url': 'http://www.zerohedge.com/news/2016-08-11/justice-department-prevented-fbi-probe-clinton-foundation-reporters-slam-state-depar', 'display_url': 'zerohedge.com/news/2016-08-1…', 'indices': [107, 130]}], 'user_mentions': [{'screen_name': 'BarackObama', 'name': 'Barack Obama', 'id': 813286, 'id_str': '813286', 'indices': [0, 12]}, {'screen_name': 'FBI', 'name': 'FBI', 'id': 17629860, 'id_str': '17629860', 'indices': [14, 18]}, {'screen_name': 'LorettaLynch', 'name': 'AG Loretta Lynch', 'id': 3290070855, 'id_str': '3290070855', 'indices': [19, 32]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [75, 91]}], 'symbols': []}",NaN,NaN
1,1,764039812479225856,he will do in one year all the things you should have done in eight,@HillaryClinton he will do in one year all the things you should have done in eight,"{'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'HillaryClinton', 'name': 'Hillary Clinton', 'id': 1339835893, 'id_str': '1339835893', 'indices': [0, 15]}], 'symbols': []}",7.626233e+17,NaN
2,2,764039849850482689,clear deliberately throwing this racein 2007 he knew that and destabilization of Mideast started wIraq invasion,"#CNN #newday clear #Trump deliberately throwing this race,in 2007 he knew that #ISIS and destabilization of Mideast started w/Iraq invasion","{'hashtags': [{'text': 'CNN', 'indices': [0, 4]}, {'text': 'newday', 'indices': [5, 12]}, {'text': 'Trump', 'indices': [19, 25]}, {'text': 'ISIS', 'indices': [79, 84]}], 'urls': [], 'user_mentions': [], 'symbols': []}",NaN,NaN
3,3,764039917924069376,you wouldnt recognize a lie if it came from your own mouth and they do continually NeverTrump,"@realDonaldTrump, you wouldn't recognize a lie if it came from your own mouth, and they do continually. #NeverTrump https://t.co/pKSQM8yikm","{'hashtags': [{'text': 'NeverTrump', 'indices': [104, 115]}], 'urls': [{'url': 'https://t.co/pKSQM8yikm', 'expanded_url': 'https://twitter.com/realdonaldtrump/status/763950742755434498', 'display_url': 'twitter.com/realdonaldtrum…', 'indices': [116, 139]}], 'user_mentions': [{'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [0, 16]}], 'symbols': []}",NaN,7.639507e+17
4,4,764039926161604608,Kid you know suing someone Thats the most beautiful thing 1 human being could do to another human being s,"""Kid, you know, suing someone? Thats the most beautiful thing 1 human being could do to another human being"" @funnyordie @realDonaldTrump😂💩s","{'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'funnyordie', 'name': 'Funny Or Die', 'id': 15693493, 'id_str': '15693493', 'indices': [109, 120]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [121, 137]}], 'symbols': []}",NaN,NaN


In [48]:
# Create a new coloumn that contains a list with all the mentions used in the tweet
def mentionfinder(row):
    text = row['text'].replace('@'," @")
    mentions = re.findall('(?<![@\w])@(\w{1,25})', text)
    mentions = [x.lower() for x in mentions]
    #mentions = tuple(mentions)
    return mentions

tweets['Mentions'] = tweets.apply(lambda row: mentionfinder(row), axis=1)

In [49]:
# Create a new coloumn that contains a list with all the hastags used in the tweet
def hashtagfinder(row):
    text = row['text'].replace('#'," #")
    hashtags = re.findall('(?:(?<=\s)|(?<=^))#.*?(?=\s|$|\'|\-|[.,\/#!$%\^&\*;:{}=\-_`~()])', text) 
    hashtags = [x.lower() for x in hashtags]
    return hashtags

tweets['Hashtags'] = tweets.apply(lambda row: hashtagfinder(row), axis=1)

In [50]:
tweets.head()

,ID,id,CleanText,text,entities,in_reply_to_status_id,quoted_status_id,Mentions,Hashtags
0,0,764039733076897792,\n\n\nALL IN COLLUSION TOGETHER \n\n\n\n\n\n\nhttpstco5GMNZq40V3,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3,"{'hashtags': [{'text': 'NOJUSTICE', 'indices': [62, 72]}, {'text': 'TrumpPence', 'indices': [93, 104]}], 'urls': [{'url': 'https://t.co/5GMNZq40V3', 'expanded_url': 'http://www.zerohedge.com/news/2016-08-11/justice-department-prevented-fbi-probe-clinton-foundation-reporters-slam-state-depar', 'display_url': 'zerohedge.com/news/2016-08-1…', 'indices': [107, 130]}], 'user_mentions': [{'screen_name': 'BarackObama', 'name': 'Barack Obama', 'id': 813286, 'id_str': '813286', 'indices': [0, 12]}, {'screen_name': 'FBI', 'name': 'FBI', 'id': 17629860, 'id_str': '17629860', 'indices': [14, 18]}, {'screen_name': 'LorettaLynch', 'name': 'AG Loretta Lynch', 'id': 3290070855, 'id_str': '3290070855', 'indices': [19, 32]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [75, 91]}], 'symbols': []}",NaN,NaN,"[barackobama, fbi, lorettalynch, realdonaldtrump]","[#nojustice, #trumppence]"
1,1,764039812479225856,he will do in one year all the things you should have done in eight,@HillaryClinton he will do in one year all the things you should have done in eight,"{'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'HillaryClinton', 'name': 'Hillary Clinton', 'id': 1339835893, 'id_str': '1339835893', 'indices': [0, 15]}], 'symbols': []}",7.626233e+17,NaN,[hillaryclinton],[]
2,2,764039849850482689,clear deliberately throwing this racein 2007 he knew that and destabilization of Mideast started wIraq invasion,"#CNN #newday clear #Trump deliberately throwing this race,in 2007 he knew that #ISIS and destabilization of Mideast started w/Iraq invasion","{'hashtags': [{'text': 'CNN', 'indices': [0, 4]}, {'text': 'newday', 'indices': [5, 12]}, {'text': 'Trump', 'indices': [19, 25]}, {'text': 'ISIS', 'indices': [79, 84]}], 'urls': [], 'user_mentions': [], 'symbols': []}",NaN,NaN,[],"[#cnn, #newday, #trump, #isis]"
3,3,764039917924069376,you wouldnt recognize a lie if it came from your own mouth and they do continually NeverTrump,"@realDonaldTrump, you wouldn't recognize a lie if it came from your own mouth, and they do continually. #NeverTrump https://t.co/pKSQM8yikm","{'hashtags': [{'text': 'NeverTrump', 'indices': [104, 115]}], 'urls': [{'url': 'https://t.co/pKSQM8yikm', 'expanded_url': 'https://twitter.com/realdonaldtrump/status/763950742755434498', 'display_url': 'twitter.com/realdonaldtrum…', 'indices': [116, 139]}], 'user_mentions': [{'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [0, 16]}], 'symbols': []}",NaN,7.639507e+17,[realdonaldtrump],[#nevertrump]
4,4,764039926161604608,Kid you know suing someone Thats the most beautiful thing 1 human being could do to another human being s,"""Kid, you know, suing someone? Thats the most beautiful thing 1 human being could do to another human being"" @funnyordie @realDonaldTrump😂💩s","{'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'funnyordie', 'name': 'Funny Or Die', 'id': 15693493, 'id_str': '15693493', 'indices': [109, 120]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [121, 137]}], 'symbols': []}",NaN,NaN,"[funnyordie, realdonaldtrump]",[]


In [51]:
len(tweets)

46811

## Cleansing the tagged train data
The untagged dataset is already cleansed in the Preprocessing.ipynb
* Removal of http and www links
* Removal of hashtags #
* Removal of multiple spaces

In addition we delete enter and tab encoding. And words that are smaller than 3 letters.

In [52]:
def clean_string(row):
    # remove http and www links from string
    rem_hl = re.sub(r'(http\S+) | (www\S+) | (https\S+)', '', row['text'])
    
    # remove @.. from string
    rem_at_hl = re.sub(r'@(\w+) |@(\w+)', '', rem_hl)
    
    # remove hashtags #.. from string
    rem_hash_at_hl = re.sub(r'#(\w+) |@(\w+)', '', rem_at_hl)

    rem_inter = re.sub(r'[^\w\s]','', rem_hash_at_hl)
    
    # remove multiple spaces
    rem_space = re.sub(' +',' ',rem_inter)
    #print (re.search('[a-zA-Z]', rem_space))
    if (re.search('[a-zA-Z]', rem_space) == None):
        return("emptyStringRetured")
    else:
        return (rem_space)


In [53]:
df['text'] = df.apply(lambda row: clean_string(row), axis=1)

In [54]:
tweets['CleanText'] = tweets['CleanText'].replace('\n','')
tweets['CleanText'] = tweets['CleanText'].replace('\t','')

df['text'] = df['text'].replace('\n','')
df['text'] = df['text'].replace('\t','')

tweets['CleanText'] = tweets['CleanText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
df['text'] = df['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))


tweets['CleanText'] =tweets['CleanText'].str.lower()
df['text'] = df['text'].str.lower()

In [55]:
np.sum(df.isnull().any(axis=1))
df.isnull().any(axis=0)

sentiment    False
text         False
dtype: bool

In [56]:
## This chuck is ment for further exploration
# df.isnull().any(axis=0)
# np.sum(df.isnull().any(axis=1))
# tweets.isnull().any(axis=0)
# np.sum(tweets.isnull().any(axis=1))
df.head() 
tweets.head()

,ID,id,CleanText,text,entities,in_reply_to_status_id,quoted_status_id,Mentions,Hashtags
0,0,764039733076897792,collusion together httpstco5gmnzq40v3,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3,"{'hashtags': [{'text': 'NOJUSTICE', 'indices': [62, 72]}, {'text': 'TrumpPence', 'indices': [93, 104]}], 'urls': [{'url': 'https://t.co/5GMNZq40V3', 'expanded_url': 'http://www.zerohedge.com/news/2016-08-11/justice-department-prevented-fbi-probe-clinton-foundation-reporters-slam-state-depar', 'display_url': 'zerohedge.com/news/2016-08-1…', 'indices': [107, 130]}], 'user_mentions': [{'screen_name': 'BarackObama', 'name': 'Barack Obama', 'id': 813286, 'id_str': '813286', 'indices': [0, 12]}, {'screen_name': 'FBI', 'name': 'FBI', 'id': 17629860, 'id_str': '17629860', 'indices': [14, 18]}, {'screen_name': 'LorettaLynch', 'name': 'AG Loretta Lynch', 'id': 3290070855, 'id_str': '3290070855', 'indices': [19, 32]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [75, 91]}], 'symbols': []}",NaN,NaN,"[barackobama, fbi, lorettalynch, realdonaldtrump]","[#nojustice, #trumppence]"
1,1,764039812479225856,will year things should have done eight,@HillaryClinton he will do in one year all the things you should have done in eight,"{'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'HillaryClinton', 'name': 'Hillary Clinton', 'id': 1339835893, 'id_str': '1339835893', 'indices': [0, 15]}], 'symbols': []}",7.626233e+17,NaN,[hillaryclinton],[]
2,2,764039849850482689,clear deliberately throwing this racein 2007 knew that destabilization mideast started wiraq invasion,"#CNN #newday clear #Trump deliberately throwing this race,in 2007 he knew that #ISIS and destabilization of Mideast started w/Iraq invasion","{'hashtags': [{'text': 'CNN', 'indices': [0, 4]}, {'text': 'newday', 'indices': [5, 12]}, {'text': 'Trump', 'indices': [19, 25]}, {'text': 'ISIS', 'indices': [79, 84]}], 'urls': [], 'user_mentions': [], 'symbols': []}",NaN,NaN,[],"[#cnn, #newday, #trump, #isis]"
3,3,764039917924069376,wouldnt recognize came from your mouth they continually nevertrump,"@realDonaldTrump, you wouldn't recognize a lie if it came from your own mouth, and they do continually. #NeverTrump https://t.co/pKSQM8yikm","{'hashtags': [{'text': 'NeverTrump', 'indices': [104, 115]}], 'urls': [{'url': 'https://t.co/pKSQM8yikm', 'expanded_url': 'https://twitter.com/realdonaldtrump/status/763950742755434498', 'display_url': 'twitter.com/realdonaldtrum…', 'indices': [116, 139]}], 'user_mentions': [{'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [0, 16]}], 'symbols': []}",NaN,7.639507e+17,[realdonaldtrump],[#nevertrump]
4,4,764039926161604608,know suing someone thats most beautiful thing human being could another human being,"""Kid, you know, suing someone? Thats the most beautiful thing 1 human being could do to another human being"" @funnyordie @realDonaldTrump😂💩s","{'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'funnyordie', 'name': 'Funny Or Die', 'id': 15693493, 'id_str': '15693493', 'indices': [109, 120]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [121, 137]}], 'symbols': []}",NaN,NaN,"[funnyordie, realdonaldtrump]",[]


## Word counts in labeld data

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer()
cvec.fit(df.text)

neg_doc_matrix = cvec.transform(df[df.sentiment == 0].text)
pos_doc_matrix = cvec.transform(df[df.sentiment == 1].text)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))
term_freq_df = pd.DataFrame([neg,pos],columns=cvec.get_feature_names()).transpose()

In [58]:
term_freq_df.columns = ['negative', 'positive']
term_freq_df['total'] = term_freq_df['negative'] + term_freq_df['positive']
term_freq_df.sort_values(by='total', ascending=False).iloc[:10]

,negative,positive,total
have,82462,62039,144501
that,72583,68739,141322
just,63178,61874,125052
with,49919,64926,114845
this,52087,40890,92977
good,28551,60840,89391
like,40639,37106,77745
dont,44916,22009,66925
your,20752,44244,64996
today,36189,28400,64589


## Sentiment analyse --Step 1 splitting the data

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['sentiment'].values, test_size=0.2)

## Sentiment analyse -- Step 2 tokanize

In [60]:
import sklearn
vect = CountVectorizer() ## This is the tokanizer
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)


In [61]:
tf_train ##1280k

<1280000x397109 sparse matrix of type '<class 'numpy.int64'>'
	with 8560082 stored elements in Compressed Sparse Row format>

In [62]:
tf_test ##320k

<320000x397109 sparse matrix of type '<class 'numpy.int64'>'
	with 2066436 stored elements in Compressed Sparse Row format>

In [63]:
## This chunk is ment to check if the porportions are still right
sum(y_train) ## 640107   is the half of 1280 k in the train set
sum(y_test) ## 159893   is the half of 320k in the test set

159688

## Sentiment analyse -- Step 3 Building a classifier

In [64]:
# p = sum of all feature count vectors with label 1
p = tf_train[y_train==1].sum(0) + 1
# q = sum of all feature count vectors with label 0
q = tf_train[y_train==0].sum(0) + 1
# Notice that we add 1 to both count vectors to ensure that every token appear at least one time in each class.

# The log-count ratio r is:
r = np.log((p/p.sum()) / (q/q.sum()))
# And b:
b = np.log(len(p) / len(q))

# Predictions

# With calculated coefficients we can now generate predictions on test set. Since we are trying to fit a linear classifier, the form of the linear equation is:

# y = mx + b
pre_preds = tf_test @ r.T + b
preds = pre_preds.T > 0
accuracy = (preds == y_test).mean()
accuracy


0.758775

The accuracy is 75.8 % meaning that the algorithm classisfy the correct label in 3 out of the 44 labels.

In [65]:
## Visualize the data
test_outcome = pd.DataFrame({'text':X_test, 'Positive':np.asarray(preds)[0]})
test_outcome.head(15)

,text,Positive
0,shouldnt have made steak dinner with sore tooth,False
1,nice mine just hard shave,True
2,ughhhh miss ashleigh have worst luck fucking worlllld,False
3,hurry want shopping milton keynes gooooooo,False
4,twitter isnt very confusing also know twitter,False
5,things feel like this dont ever want come downthink might have liked thatwe could have hung onto balloons,False
6,miss sister,False
7,lovin sunshine shame only shines about once year glasgow,True
8,well lets hasty need rush these things good afternoon sirrah,True
9,bring back some mambo sauce,True


## Sentiment analyse -- Step 4 Score out on our dataset

In [66]:
# Vectorize the unseen data
unseen = vect.transform(tweets.CleanText)
pre_unseen = unseen @ r.T + b
predictions_unseen = pre_unseen.T > 0

In [67]:
##Visualize the data
pd.options.display.max_rows  ## This is for showing all the text in the column (otherwise it will be half....)
pd.set_option('display.max_colwidth', -1)

Predictions_unseen_df = pd.DataFrame({'text':tweets.text, 'Positive':np.asarray(predictions_unseen)[0]})
Predictions_unseen_df.head(20)


,text,Positive
0,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3,True
1,@HillaryClinton he will do in one year all the things you should have done in eight,False
2,"#CNN #newday clear #Trump deliberately throwing this race,in 2007 he knew that #ISIS and destabilization of Mideast started w/Iraq invasion",False
3,"@realDonaldTrump, you wouldn't recognize a lie if it came from your own mouth, and they do continually. #NeverTrump https://t.co/pKSQM8yikm",False
4,"""Kid, you know, suing someone? Thats the most beautiful thing 1 human being could do to another human being"" @funnyordie @realDonaldTrump😂💩s",True
5,"@HillaryClinton you ARE the co-founder of ISIS, you crooked, evil, lying, witch. How can you live with yourself?",True
6,@Geraldanthro @NeilTurner_ @realDonaldTrump want to do a comparison try maimed Vets pre &amp; post Iraq pullout. Bar graph that. @washingtonpost,False
7,@mike4193496 @realDonaldTrump I TOTALLY CONCUR!! This Election is just CRA CRA n Corruption in our Gov is Mind Blowing!! Trump= Last Hope!!!,False
8,"@realDonaldTrump @elsolarverde What issues? Your idiot claim that she ""founded"" ISIS? Trump will go to Hell for lying &amp; stealing. Shame!",False
9,Can't stand @HillaryClinton or @realDonaldTrump?Take a look. They can win...don't settle. #15for15 #TeamGov #YouIn https://t.co/YK336aaH98,False


In [68]:
## This is a check if we have still all the data (46811) in the dataframe
len(Predictions_unseen_df) #46811
sum(Predictions_unseen_df.Positive) ## Amount of positive == 24217

24266

## Write back the dataset to share

In [69]:
# tweets = pd.read_json("C:\\Users\\Jeffr\\Downloads\\twitter_geotagged_clean.json", orient='records', lines=True)
# tweets["Positive"] = np.asarray(predictions_unseen)[0]
# tweets.head()
tweets2 = tweets
# tweets.to_json('C:\\Users\\Jeffr\\Downloads\\twitter_geotagged_clean_including_sentiment.json', orient='records', lines=True)

# Vanaf hier begint de selectie op trump of hillary

In [70]:
full = pd.merge(Predictions_unseen_df,tweets2, how='left', left_on='text', right_on='text')

In [71]:
full.head()

,text,Positive,ID,id,CleanText,entities,in_reply_to_status_id,quoted_status_id,Mentions,Hashtags
0,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3,True,0,764039733076897792,collusion together httpstco5gmnzq40v3,"{'hashtags': [{'text': 'NOJUSTICE', 'indices': [62, 72]}, {'text': 'TrumpPence', 'indices': [93, 104]}], 'urls': [{'url': 'https://t.co/5GMNZq40V3', 'expanded_url': 'http://www.zerohedge.com/news/2016-08-11/justice-department-prevented-fbi-probe-clinton-foundation-reporters-slam-state-depar', 'display_url': 'zerohedge.com/news/2016-08-1…', 'indices': [107, 130]}], 'user_mentions': [{'screen_name': 'BarackObama', 'name': 'Barack Obama', 'id': 813286, 'id_str': '813286', 'indices': [0, 12]}, {'screen_name': 'FBI', 'name': 'FBI', 'id': 17629860, 'id_str': '17629860', 'indices': [14, 18]}, {'screen_name': 'LorettaLynch', 'name': 'AG Loretta Lynch', 'id': 3290070855, 'id_str': '3290070855', 'indices': [19, 32]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [75, 91]}], 'symbols': []}",NaN,NaN,"[barackobama, fbi, lorettalynch, realdonaldtrump]","[#nojustice, #trumppence]"
1,@HillaryClinton he will do in one year all the things you should have done in eight,False,1,764039812479225856,will year things should have done eight,"{'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'HillaryClinton', 'name': 'Hillary Clinton', 'id': 1339835893, 'id_str': '1339835893', 'indices': [0, 15]}], 'symbols': []}",7.626233e+17,NaN,[hillaryclinton],[]
2,"#CNN #newday clear #Trump deliberately throwing this race,in 2007 he knew that #ISIS and destabilization of Mideast started w/Iraq invasion",False,2,764039849850482689,clear deliberately throwing this racein 2007 knew that destabilization mideast started wiraq invasion,"{'hashtags': [{'text': 'CNN', 'indices': [0, 4]}, {'text': 'newday', 'indices': [5, 12]}, {'text': 'Trump', 'indices': [19, 25]}, {'text': 'ISIS', 'indices': [79, 84]}], 'urls': [], 'user_mentions': [], 'symbols': []}",NaN,NaN,[],"[#cnn, #newday, #trump, #isis]"
3,"@realDonaldTrump, you wouldn't recognize a lie if it came from your own mouth, and they do continually. #NeverTrump https://t.co/pKSQM8yikm",False,3,764039917924069376,wouldnt recognize came from your mouth they continually nevertrump,"{'hashtags': [{'text': 'NeverTrump', 'indices': [104, 115]}], 'urls': [{'url': 'https://t.co/pKSQM8yikm', 'expanded_url': 'https://twitter.com/realdonaldtrump/status/763950742755434498', 'display_url': 'twitter.com/realdonaldtrum…', 'indices': [116, 139]}], 'user_mentions': [{'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [0, 16]}], 'symbols': []}",NaN,7.639507e+17,[realdonaldtrump],[#nevertrump]
4,"""Kid, you know, suing someone? Thats the most beautiful thing 1 human being could do to another human being"" @funnyordie @realDonaldTrump😂💩s",True,4,764039926161604608,know suing someone thats most beautiful thing human being could another human being,"{'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'funnyordie', 'name': 'Funny Or Die', 'id': 15693493, 'id_str': '15693493', 'indices': [109, 120]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [121, 137]}], 'symbols': []}",NaN,NaN,"[funnyordie, realdonaldtrump]",[]


In [72]:
Hillary = ['#dumptrump','#nevertrump', '#imwithher']
Neutral = ['#trumppence16', '#hillaryclinton', '#hillary', '#donaldtrump', '#trump']
NeutralTrump = ['#trumppence16', '#donaldtrump', '#trump', '#donaldtrump\'s', '#trump\'s']
NeutralHillary = ['#hillaryclinton', '#hillary', '#hillaryclinton\'s']
Trump = ['#maga', '#crookedhillary', '#neverhillary']
allhash = ['@HillaryClinton', '#maga', '#trumppence16', '#hillaryclinton', '#hillary', '#crookedhillary', '#donaldtrump', '#dumptrump', '@realDonaldTrump', '#nevertrump', '#imwithher', '#neverhillary', '#trump']

In [73]:
import math

In [74]:
"""
Assumption: Everyone that tweets negative about Hillary is for Trump 
(and the other way around).
"""
def trumphillary(row):
    #print(row)
    # HASHTAGS
    # First checking if there are hashtags used that are person specific
    if len(row['Hashtags']) != 0:
        if ('#dumptrump' in row['Hashtags'] or '#nevertrump' in row['Hashtags'] or '#imwithher' in row['Hashtags']) and ('#maga' in row['Hashtags'] or '#crookedhillary' in row['Hashtags'] or '#neverhillary' in row['Hashtags']):
            return 'Both'
        for i in range(len(row['Hashtags'])):
            if row['Hashtags'][i] in Hillary:
                return 'Hillary'
            elif row['Hashtags'][i] in Trump:
                return 'Trump'
            else:
                continue

    # MENTIONS        
    # Hillary is returned when Hillary is mentioned and something positive 
    # is said or if Trump is mentioned and something negative is said and 
    # Trump the other way around
    if len(row['Mentions']) != 0: 
        if 'HillaryClinton' in row['Mentions'] and 'realDonaldTrump' in row['Mentions']:
                return 'Both'
        for j in range(len(row['Mentions'])):
            if row['Mentions'][j] == "hillaryclinton":
                if row['Positive'] == True:
                    return 'Hillary'
                elif row['Positive'] == False:
                    return 'Trump'
                else:
                    continue
            elif row['Mentions'][j] == "realdonaldtrump":
                if row['Positive'] == True:
                    return 'Trump'
                elif row['Positive'] == False:
                    return 'Hillary'
                else:
                    continue
            else:
                continue
            
    
    # COMBINED
    if len(row['Hashtags']) != 0:
        #print(row)
        for i in range(len(row['Hashtags'])):
            if row['Hashtags'][i] in NeutralHillary:
                if row['Positive'] == True:
                    return 'Hillary'
                elif row['Positive'] == False:
                    return 'Trump'
                else:
                    continue
            if row['Hashtags'][i] in NeutralTrump:
                if row['Positive'] == True:
                    return 'Trump'
                elif row['Positive'] == False:
                    return 'Hillary'
                else:
                    continue
    
    if len(row['Hashtags']) != 0:
        for hashtag in (row['Hashtags']):
            if re.search('trump', hashtag):
                if row['Positive'] == True:
                    return 'Trump'
                elif row['Positive'] == False:
                    return 'Hillary'
                else:
                    continue

    if len(row['Hashtags']) != 0:
        for hashtag in (row['Hashtags']):
            if re.search('hillary', hashtag) or re.search('clinton', hashtag):
                if row['Positive'] == True:
                    return 'hillary'
                elif row['Positive'] == False:
                    return 'trump' 
                else:
                    continue
    
    # IF MENTIONS AND HASHTAGS AREN'T CLEAR ENOUGH
    
    if math.isnan(row['quoted_status_id']) == False:
        return 'Quoted'
    elif math.isnan(row['in_reply_to_status_id']) == False:
        return 'Reply'
    else:
        return 'Else'
full['For'] = full.apply(lambda row: trumphillary(row), axis=1)

In [75]:
len(full)

48125

In [76]:
full[full['For']=='Else']

,text,Positive,ID,id,CleanText,entities,in_reply_to_status_id,quoted_status_id,Mentions,Hashtags,For
297,"Hey #ImWithHer🇺🇸👠, and we're making ""Herstory"" together because we're #StrongerTogether!… https://t.co/HE2lYITEfb",True,296,764051205215571968,imwithher were making herstory together because were strongertogether,"{'hashtags': [{'text': 'ImWithHer', 'indices': [4, 14]}, {'text': 'StrongerTogether', 'indices': [70, 87]}], 'urls': [{'url': 'https://t.co/HE2lYITEfb', 'expanded_url': 'https://www.instagram.com/p/BJAYIA_g7r5/', 'display_url': 'instagram.com/p/BJAYIA_g7r5/', 'indices': [90, 113]}], 'user_mentions': [], 'symbols': []}",NaN,NaN,[],"[#imwithher🇺🇸👠, #strongertogether]",Else
10690,"This Dem is ""extra special""! Have a seat &amp; listen sir. Let this officer speak the truth! #MAGA🇺🇸🇺🇸 https://t.co/5XCc7wYHPS via @youtube",True,10518,764181358776201216,this extra special have seat listen this officer speak truth maga,"{'hashtags': [{'text': 'MAGA', 'indices': [93, 98]}], 'urls': [{'url': 'https://t.co/5XCc7wYHPS', 'expanded_url': 'http://www.youtube.com/watch?v=C9O8cvm6wko&sns=tw', 'display_url': 'youtube.com/watch?v=C9O8cv…', 'indices': [103, 126]}], 'user_mentions': [{'screen_name': 'YouTube', 'name': 'YouTube', 'id': 10228272, 'id_str': '10228272', 'indices': [131, 139]}], 'symbols': []}",NaN,NaN,[youtube],[#maga🇺🇸🇺🇸],Else
11002,https://t.co/7DU3by37Ho &amp; #putin are pedophiles,False,10830,764185722446352384,pedophiles,"{'hashtags': [{'text': 'putin', 'indices': [30, 36]}], 'urls': [{'url': 'https://t.co/7DU3by37Ho', 'expanded_url': 'http://www.snopes.com/2016/06/23/donald-trump-rape-lawsuit/#trump', 'display_url': 'snopes.com/2016/06/23/don…', 'indices': [0, 23]}], 'user_mentions': [], 'symbols': []}",NaN,NaN,[],[#putin],Else
17302,https://t.co/ZXAugPjrLj,False,16904,764261115639377921,httpstcozxaugpjrlj,"{'hashtags': [], 'urls': [{'url': 'https://t.co/ZXAugPjrLj', 'expanded_url': 'https://youtu.be/8EDhBRa0f00#trump', 'display_url': 'youtu.be/8EDhBRa0f00#tr…', 'indices': [0, 23]}], 'user_mentions': [], 'symbols': []}",NaN,NaN,[],[],Else
19682,Preach sisters! #yasqween #badassbitches #hrc #imwithher… https://t.co/mcZtGlScm7,True,19242,764290374537998336,preach sisters imwithher,"{'hashtags': [{'text': 'yasqween', 'indices': [16, 25]}, {'text': 'badassbitches', 'indices': [26, 40]}, {'text': 'hrc', 'indices': [41, 45]}, {'text': 'imwithher', 'indices': [46, 56]}], 'urls': [{'url': 'https://t.co/mcZtGlScm7', 'expanded_url': 'https://www.instagram.com/p/BJCE4vzjVaGa5LsLUFDlzKO4dZRABzxGSL-GAA0/', 'display_url': 'instagram.com/p/BJCE4vzjVaGa…', 'indices': [58, 81]}], 'user_mentions': [], 'symbols': []}",NaN,NaN,[],"[#yasqween, #badassbitches, #hrc, #imwithher…]",Else
20670,https://t.co/LbXHp4N0cf I don't think you meant to delete. Wanted to make sure you got it back. Anything to help get ur message 2 EVERYONE!,False,20200,764307794447175680,dont think meant delete wanted make sure back anything help message everyone,"{'hashtags': [], 'urls': [{'url': 'https://t.co/LbXHp4N0cf', 'expanded_url': 'http://twitter.com/SaysSheToday/status/763898399393480704/photo/1.@realDonaldTrump', 'display_url': 'twitter.com/SaysSheToday/s…', 'indices': [0, 23]}], 'user_mentions': [], 'symbols': []}",NaN,NaN,[],[],Else
21333,"Hillary Caught on Video ADMITTING She Helped Obama Create ISIS…Trump was Right! - https://t.co/Hg4QVT8RqJ You are correct, per HRC @greta ??",True,20860,764321162830684160,hillary caught video admitting helped obama create isistrump right correct,"{'hashtags': [], 'urls': [{'url': 'https://t.co/Hg4QVT8RqJ', 'expanded_url': 'http://www.proudcons.com/hillary-caught-on-video-admitting-she-helped-obama-create-isis-trump-was-right/@realDonaldTrump', 'display_url': 'proudcons.com/hillary-caught…', 'indices': [82, 105]}], 'user_mentions': [{'screen_name': 'greta', 'name': 'Greta Van Susteren', 'id': 16031927, 'id_str': '16031927', 'indices': [131, 137]}], 'symbols': []}",NaN,NaN,[greta],[],Else
40003,Prayers for #Milwauke

In [77]:
full.For.value_counts()

Trump      21816
Hillary    21526
Quoted     4590 
Both       93   
trump      46   
hillary    42   
Else       9    
Reply      3    
Name: For, dtype: int64

In [78]:
full.For.value_counts()

Trump      21816
Hillary    21526
Quoted     4590 
Both       93   
trump      46   
hillary    42   
Else       9    
Reply      3    
Name: For, dtype: int64

In [79]:
full.in_reply_to_status_id

0       NaN          
1        7.626233e+17
2       NaN          
3       NaN          
4       NaN          
5       NaN          
6        7.640372e+17
7        7.640015e+17
8        7.639497e+17
9       NaN          
10       7.639930e+17
11       7.639060e+17
12      NaN          
13       7.626233e+17
14      NaN          
15       7.639535e+17
16       7.639507e+17
17      NaN          
18       7.637870e+17
19       7.639060e+17
20      NaN          
21      NaN          
22       7.639507e+17
23      NaN          
24       7.639511e+17
25      NaN          
26      NaN          
27      NaN          
28      NaN          
29      NaN          
         ..          
48095    7.648800e+17
48096    7.647999e+17
48097    7.648738e+17
48098    7.648786e+17
48099   NaN          
48100    7.648669e+17
48101    7.648708e+17
48102    7.648669e+17
48103   NaN          
48104    7.648723e+17
48105    7.648329e+17
48106   NaN          
48107   NaN          
48108    7.647999e+17
48109    7

In [80]:
full.For.value_counts()

Trump      21816
Hillary    21526
Quoted     4590 
Both       93   
trump      46   
hillary    42   
Else       9    
Reply      3    
Name: For, dtype: int64

In [81]:
for i in range(3):
    print(i)

0
1
2
